In [0]:
df = spark.table("gizmobox.bronze.py_orders")
df.display()

In [0]:
%sql
create or replace temporary view tv_orders
as
select 
regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})','"order_date": "\$1"') as new_value
from gizmobox.bronze.v_orders

In [0]:
%sql
select schema_of_json(new_value) from tv_orders limit 1;

In [0]:
%sql
drop table if exists gizmobox.silver.orders_json;
create table if not exists gizmobox.silver.orders_json
as
select 
from_json(
  new_value
  ,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as fixed_value
from tv_orders

In [0]:
%sql
select * from gizmobox.silver.orders_json